In [1]:
import ipdb


In [2]:
# FMKS version
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from dask.multiprocessing import get
import dask.array as da

import numpy as np
n_states = 3
domain = [-1, 1]

from pymks.bases import LegendreBasis
from pymks.fmks.data.cahn_hilliard import generate, solve
leg_basis = LegendreBasis(n_states=n_states, domain=domain)
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
reducer = PCA(n_components=3)
linker = LogisticRegression()


In [3]:
da.random.seed(99)

data = generate(shape=(100, 41, 41), chunks=(40, 41, 41), delta_t=1e-2)

In [4]:
X0,X1= data
y0 = np.zeros(X0.shape[0])
y1 = np.ones(X1.shape[0])
X = np.concatenate((X0, X1))
y = np.concatenate((y0, y1))

In [5]:
# My own Primitiva bases
# replaces Primitive transform nstate 2 min -1 max 1

def Primitivebin(X):
    
    X[X>0]=0
    X[X<0]=1
    return X

In [6]:
from pymks.fmks.correlations import auto_correlation

In [7]:

chunks=X.shape
Xa= da.from_array(X, chunks=chunks)
Stats=auto_correlation(Xa)

In [8]:
FlattenStats=da.Array.flatten(Stats).reshape(200,(41*41))
pca = PCA(n_components=3)
pca.fit(FlattenStats)


PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [9]:
Scores=pca.transform(FlattenStats)


In [10]:
lf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(Scores, y)


In [11]:
## Create the test set 

datatest = generate(shape=(3, 41, 41), chunks=(1, 41, 41), delta_t=1e-2)

X0_test,X1_test= datatest


X0_test=Primitivebin(X0_test)
X1_test=Primitivebin(X1_test)


In [12]:
chunks1=X0_test.shape
X1= da.from_array(X0_test, chunks=chunks1)
testStats1=auto_correlation(X1)
FlattenStats1=da.Array.flatten(testStats1).reshape(3,(41*41))
chunks2=X1_test.shape
X2= da.from_array(X1_test, chunks=chunks2)
testStats2=auto_correlation(X2)
FlattenStats2=da.Array.flatten(testStats2).reshape(3,(41*41))



test1_pc=pca.transform(FlattenStats1)
test2_pc=pca.transform(FlattenStats2)




y0_test = lf.predict(test1_pc)
y1_test = lf.predict(test2_pc)
assert np.allclose(y0_test, [0, 0, 0])
assert np.allclose(y1_test, [1, 1, 1])




array([0., 0., 0.])

array([1., 1., 1.])